In [140]:
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
import re

In [13]:
url = "https://qudobeauty.com/"
cleansing_url = "https://qudobeauty.com/cat/wholesale-face-care/cleanser/"

In [68]:
website = requests.get(url)
homepage_soup = BeautifulSoup(website.text, "html.parser")

### Get cleansing products

In [17]:
cleansing_page = requests.get(cleansing_url)

In [32]:
cleansing_page_soup = BeautifulSoup(cleansing_page.text, "html.parser")

In [36]:
cleansing_products = cleansing_page_soup.select("ul.products li.product")

In [49]:
cleansing_products[0]

<li class="product type-product post-401 status-publish first instock product_cat-exfoliants product_cat-cleanser has-post-thumbnail taxable shipping-taxable purchasable product-type-simple">
<div class="commercekit-wishlist mini" data-product-id="401"><a aria-label="Wishlist" class="commercekit-save-wishlist" data-product-id="401" data-type="list" href="#" title="Wishlist"><em class="cg-wishlist"></em><span></span></a></div><div class="woocommerce-image__wrapper"><a class="woocommerce-LoopProduct-link woocommerce-loop-product__link" href="https://qudobeauty.com/product/anua-heartleaf-lha-moisture-peeling-gel-120ml/" title="Anua – Heartleaf LHA Moisture Peeling Gel, 120ml"><div class="shoptimizer-plp-carousel-container"><div class="shoptimizer-plp-image-wrapper"><div class="pbfw_square_data_main"><img alt="Anua - Heartleaf LHA Moisture Peeling Gel, 120ml" class="attachment-woocommerce_thumbnail size-woocommerce_thumbnail" decoding="async" height="300" loading="lazy" sizes="(max-width: 

In [74]:
# get individual cleansing products
item_cap = 30
products = []

for index in range(item_cap):
    product = cleansing_products[index]
    link = product.select_one(".woocommerce-LoopProduct-link")
    img = product.select_one("img")

    products.append({
        "name": link["title"] if link else None,
        "product_url": link["href"] if link else None,
        "image": img["src"] if img else None,
        "brand": link["title"].split("–")[0].strip() if link else None,
        "categories": [
            cat.replace("product_cat-", "")
            for cat in product.get("class", [])
            if cat.startswith("product_cat-")
        ],
        "ingredients": [],
        "size": None
    })

In [82]:
products[0]

{'name': 'Anua – Heartleaf LHA Moisture Peeling Gel, 120ml',
 'product_url': 'https://qudobeauty.com/product/anua-heartleaf-lha-moisture-peeling-gel-120ml/',
 'image': 'https://qudobeauty.com/wp-content/uploads/2025/04/anua-us-cleanser-200ml-heartleaf-lha-moisture-peeling-gel-1161189356-300x300.webp',
 'brand': 'Anua',
 'categories': ['exfoliants', 'cleanser'],
 'ingredients': [],
 'size': None}

In [153]:
def extract_product_ingredients_and_capacity(soup):

    def extract_capacity(text):
        if "capacity" in text.lower():
            cap = p.get_text(" ", strip=True)
            match = re.search(r"Capacity:\s*([^A-Z]+)", cap)
            if match:
                return match.group(1).strip()

    ingredients = []
    capacity = None
    description = soup.select_one(".woocommerce-Tabs-panel--description")

    paragraphs = description.find_all("p")

    # ingredients are listed between "Product contains" and "Product effects"
    capture = False
    for p in paragraphs:

        strong = p.find("strong")

        if not strong:
            continue

        text = strong.get_text(strip=True)

        capacity = extract_capacity(text)

        # start capturing
        if "product contains" in text.lower():
            capture = True
            continue

        # stop capturing
        if "product effects" in text.lower():
            capture = False

        if capture:
            ingredients.append(text)

    return {
        "ingredients": ingredients,
        "capacity": capacity
    }

In [155]:
# visit each product url and retrieve remaining data (ingredients etc)

for product in products:
    if product["product_url"] is not None:
        single_product_page = requests.get(product["product_url"])
        single_product_soup = BeautifulSoup(single_product_page.text, "html.parser")

        results = extract_product_ingredients_and_capacity(single_product_soup)
        
        product["ingredients"] = results["ingredients"]
        product["size"] = results["capacity"]
    

In [156]:
products

[{'name': 'Anua – Heartleaf LHA Moisture Peeling Gel, 120ml',
  'product_url': 'https://qudobeauty.com/product/anua-heartleaf-lha-moisture-peeling-gel-120ml/',
  'image': 'https://qudobeauty.com/wp-content/uploads/2025/04/anua-us-cleanser-200ml-heartleaf-lha-moisture-peeling-gel-1161189356-300x300.webp',
  'brand': 'Anua',
  'categories': ['exfoliants', 'cleanser'],
  'ingredients': ['Heartleaf (Houttuynia Cordata) Extract',
   'LHA (Lipo Hydroxy Acid)',
   'Panthenol (Vitamin B5)',
   'Allantoin',
   'Hyaluronic Acid',
   'Centella Asiatica Extract'],
  'size': '120 ml'},
 {'name': 'Anua – Rice Enzyme Brightening Cleansing Powder, 40g',
  'product_url': 'https://qudobeauty.com/product/anua-rice-enzyme-brightening-cleansing-powder-40g/',
  'image': 'https://qudobeauty.com/wp-content/uploads/2025/04/Anua-Rice-Enzyme-Brightening-Cleansing-Powder-40g-300x300.jpg',
  'brand': 'Anua',
  'categories': ['cleanser'],
  'ingredients': ['Rice Powder & Rice Enzyme Extract',
   'Papain (Enzyme fro

In [158]:
# save products as csv
import csv

with open("products.csv", 'w', newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=[
            "Prodct name", "Brand", "Category/type", "Ingredients", "Size/packaging", "Product image URL", "Product page URL"
        ]
    )

    writer.writeheader()

    for product in products:
        writer.writerow({
            "Prodct name": product["name"],
            "Brand": product["brand"],
            "Category/type": ";".join(product["categories"]),
            "Ingredients": ";".join(product["ingredients"]),
            "Size/packaging": product["size"],
            "Product image URL": product["image"],
            "Product page URL": product["product_url"]
        })

In [ ]:
# u = "https://qudobeauty.com/product/anua-heartleaf-lha-moisture-peeling-gel-120ml/"

# single_product_page = requests.get(u)
# single_product_soup = BeautifulSoup(single_product_page.text, "html.parser")

In [ ]:
# single_product_description = single_product_soup.select_one(".woocommerce-Tabs-panel--description")

In [157]:
# extract_product_ingredients_and_capacity(single_product_soup)